In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

from scipy import stats

### Acquire the Merged.csv of the health and Yelp data

In [3]:
df=pd.read_csv('data/merged.csv')
pd.set_option('display.max_columns', None)

In [4]:
df=pd.DataFrame(df)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 29 columns):
Unnamed: 0              2409 non-null int64
inspection_date         2409 non-null object
inspector               2409 non-null object
number_of_violations    2409 non-null int64
violations              2409 non-null object
zip_code                2409 non-null int64
house_numbers           2409 non-null int64
categories              2409 non-null object
transactions            2409 non-null object
distance                2409 non-null float64
review_count            2409 non-null float64
longitude               2409 non-null float64
latitude                2409 non-null float64
name                    2409 non-null object
sector                  2409 non-null int64
district                2409 non-null object
total_score             2409 non-null int64
rating                  2409 non-null float64
price                   2409 non-null object
establishment_x         2409 non-null o

In [5]:
# Dropping redundant and unneccesary columns

df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)
df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000
mean,1204.000000,4.989622,78220.360731,4494.122042,7806.578807,161.892902,-98.514704,29.486358,16.587796,94.044832,3.574720
std,695.562722,4.708035,14.884181,4491.037788,3305.930381,264.744464,0.059607,0.055306,9.608700,5.482347,0.769284
min,0.000000,0.000000,78201.000000,1.000000,811.199667,1.000000,-98.708442,29.354318,1.000000,56.000000,1.000000
25%,602.000000,2.000000,78209.000000,849.000000,5840.653743,22.000000,-98.564331,29.430632,8.000000,91.000000,3.000000
50%,1204.000000,4.000000,78216.000000,2727.000000,7940.300340,69.000000,-98.499740,29.486788,17.000000,95.000000,3.500000
75%,1806.000000,7.000000,78230.000000,7400.000000,9571.434425,186.000000,-98.480582,29.526893,22.000000,98.000000,4.000000
max,2408.000000,50.000000,78259.000000,22211.000000,20222.717722,2661.000000,-98.397780,29.661520,35.000000,100.000000,5.000000


In [6]:
#Dropped all establishments with less than 22 reviews 
# df=  df[df['review_count'] >= 22] 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 17 columns):
Unnamed: 0              2409 non-null int64
inspection_date         2409 non-null object
inspector               2409 non-null object
number_of_violations    2409 non-null int64
violations              2409 non-null object
zip_code                2409 non-null int64
house_numbers           2409 non-null int64
categories              2409 non-null object
distance                2409 non-null float64
review_count            2409 non-null float64
longitude               2409 non-null float64
latitude                2409 non-null float64
name                    2409 non-null object
sector                  2409 non-null int64
district                2409 non-null object
total_score             2409 non-null int64
rating                  2409 non-null float64
dtypes: float64(5), int64(6), object(6)
memory usage: 320.0+ KB


## Grouping Health Scores into bins

In [7]:
#grouped health scores into 4 groups
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 99:
        health_scores['health_scores'].append('1')
    elif x>=96:
        health_scores['health_scores'].append('2')
    elif x >=92:
        health_scores['health_scores'].append('3')
    else :
        health_scores['health_scores'].append('4')
#df
df['health_scores']= health_scores['health_scores']  

## Checking the distribution of health scores

In [8]:
df['health_scores'].value_counts()

2    697
4    657
3    608
1    447
Name: health_scores, dtype: int64

In [9]:
#encode for the inspectors
df['inspector'].value_counts()

Lauren Schumacher     194
Jacob Johanson        159
RYAN DAVIS            153
Jeffrey Carrizales    148
Sumner Bumgardner     127
Diana Garcia          117
Cicily Martinez       116
Bamba Njie            109
Bernard Capdeboscq    105
Valerie Suarez        104
Benito Zuniga          97
Deborah Liu            96
Nathan Balfour         90
Raul Jimenez           87
Daisy Zamora           82
Debra Hernandez        59
MICHELLE MORENO        55
Mario Cavazos          54
Kassandra Aguilar      53
Amanda Cantu           52
John Payelle           48
William Hudson         43
Erica Arreola          42
William Emminger       37
Kathleen Prenzler      34
Adan Zamora            24
Monica Caballero       22
Kacie Patrick          20
Sean Gilbert           20
Fermin Garza           18
Joe Mokry              15
Juanicia Page          10
Victoria Paredes        9
Daniel Guzman           4
Maurico Ripley          3
Erica Llanas            2
Rebecca Vera            1
Name: inspector, dtype: int64

In [10]:
#Dropping inspectors that have not copmpleted AT LEAST 2 inspections
vdf = pd.DataFrame(df['inspector'].value_counts())
vdf['check'] = vdf['inspector'] > 1
over_20 = list(vdf.loc[vdf['check'], :].reset_index()['index'])

In [11]:
df['check'] = df['inspector'].isin(over_20)
new_df = df.loc[df['check'], :]
new_df = new_df.drop(columns = ['check', 'Unnamed: 0'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2408 entries, 0 to 2408
Data columns (total 17 columns):
inspection_date         2408 non-null object
inspector               2408 non-null object
number_of_violations    2408 non-null int64
violations              2408 non-null object
zip_code                2408 non-null int64
house_numbers           2408 non-null int64
categories              2408 non-null object
distance                2408 non-null float64
review_count            2408 non-null float64
longitude               2408 non-null float64
latitude                2408 non-null float64
name                    2408 non-null object
sector                  2408 non-null int64
district                2408 non-null object
total_score             2408 non-null int64
rating                  2408 non-null float64
health_scores           2408 non-null object
dtypes: float64(5), int64(5), object(7)
memory usage: 338.6+ KB


## Encoding sector data to add geographical reference to inspector data

## Total Entries is 2408

In [12]:
#Encoding the inspector column for modeling.
label_enc= LabelEncoder()
new_df["sector_encode"] = label_enc.fit_transform(new_df.sector)


In [13]:
#OneHot Encoding
one_enc = OneHotEncoder()
Y = one_enc.fit_transform(new_df.sector_encode.values.reshape(-1,1)).toarray()
# df_array = np.array(new_long_df.encoded).reshape(len(new_long_df.encoded),1)


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [14]:
#Joining the encoded data to the main Df
# elist = [X,Y,Z]
dfOneHot = pd.DataFrame(Y, columns = ["sector_"+str(int(i)) for i in range(Y.shape[1])])

In [15]:
new_df = new_df.reset_index()
new_df = new_df.reset_index()
new_df['index']= new_df['level_0']
new_df = new_df.drop(columns = ['level_0'])
dfOneHot = dfOneHot.reset_index()
enc_df=new_df.join(dfOneHot, on = 'index', lsuffix='_left', rsuffix='_right')

### Decision Tree Model

In [16]:
#### Decision tree including number of review, total violations, yelp score, and inspector
#training / test split
X= enc_df.drop(columns=[ 'index_left', 'index_right', 'inspection_date','longitude', 'latitude',  'inspector', 'number_of_violations','violations', 'zip_code', 'house_numbers', 'categories', 'distance', 'name', 'sector', 'district', 'health_scores', 'total_score', 'sector_encode'])
y= enc_df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=enc_df.health_scores)
X_train.head()


,review_count,rating,sector_0,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,sector_10,sector_11,sector_12,sector_13,sector_14,sector_15,sector_16,sector_17,sector_18,sector_19,sector_20,sector_21,sector_22,sector_23,sector_24,sector_25,sector_26,sector_27,sector_28,sector_29,sector_30,sector_31,sector_32,sector_33
770,2509.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,148.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1333,64.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,660.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2025,9.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#Train Model

#Create the Decision Tree Object


clf = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state= 42, min_samples_split= 2)

In [18]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [19]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

array(['2', '2', '2', ..., '2', '2', '2'], dtype=object)

In [20]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.15479876, 0.29566563, 0.26083591, 0.28869969],
       [0.15479876, 0.29566563, 0.26083591, 0.28869969],
       [0.15479876, 0.29566563, 0.26083591, 0.28869969],
       ...,
       [0.15479876, 0.29566563, 0.26083591, 0.28869969],
       [0.15479876, 0.29566563, 0.26083591, 0.28869969],
       [0.15479876, 0.29566563, 0.26083591, 0.28869969]])

In [21]:
#Create a confusion matrix
confusion_matrix(y_train, y_pred)

array([[ 87, 226,   0,   0],
       [ 34, 446,   2,   5],
       [ 18, 361,  12,  34],
       [  6, 383,   0,  71]])

In [22]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.37


In [23]:
#Create a classification report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.60      0.28      0.38       313
           2       0.31      0.92      0.47       487
           3       0.86      0.03      0.05       425
           4       0.65      0.15      0.25       460

    accuracy                           0.37      1685
   macro avg       0.60      0.34      0.29      1685
weighted avg       0.59      0.37      0.29      1685



In [24]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.36


In [25]:
#Testing the accuracy with chi square

#Health scores and inspector

contingency_table = pd.crosstab(enc_df.inspector, enc_df.health_scores)
chi2_stat, p_val, dof, ex = stats.chi2_contingency(contingency_table)
print("===Chi2 Stat===")
print(chi2_stat)
print("\n")
print("===Degrees of Freedom===")
print(dof)
print("\n")
print("===P-Value===")
print(p_val)
print("\n")


===Chi2 Stat===
724.4046817297015


===Degrees of Freedom===
105


===P-Value===
3.1789639786166964e-93




### Decision Tree Model- Binning based on probability

In [26]:
#created a table to show the probability of inspectors assigning a certain health score

contingency_table = pd.crosstab(df.inspector, df.health_scores, margins=True)
contingency_table = contingency_table[contingency_table.All > 20]
contingency_table['less_92'] = round(contingency_table['4']/contingency_table['All'],2)
contingency_table['92_to_95'] = round(contingency_table['3']/contingency_table['All'],2)
contingency_table['96_to_99'] = round(contingency_table['2']/contingency_table['All'],2)
contingency_table['100'] = round(contingency_table['1']/contingency_table['All'],2)
contingency_table.reset_index(inplace = True)
contingency_table

health_scores,inspector,1,2,3,4,All,less_92,92_to_95,96_to_99,100
0,Adan Zamora,7,11,2,4,24,0.17,0.08,0.46,0.29
1,Amanda Cantu,6,19,15,12,52,0.23,0.29,0.37,0.12
2,Bamba Njie,3,32,38,36,109,0.33,0.35,0.29,0.03
3,Benito Zuniga,17,28,25,27,97,0.28,0.26,0.29,0.18
4,Bernard Capdeboscq,10,28,30,37,105,0.35,0.29,0.27,0.10
5,Cicily Martinez,17,27,37,35,116,0.30,0.32,0.23,0.15
6,Daisy Zamora,14,43,14,11,82,0.13,0.17,0.52,0.17
7,Deborah Liu,11,38,35,12,96,0.12,0.36,0.40,0.11
8,Debra Hernandez,24,23,6,6,59,0.10,0.10,0.39,0.41
9,Diana Garcia,5,21,38,53,117,0.45,0.32,0.18,0.04


In [27]:
df = enc_df.merge(contingency_table)

In [28]:
df.columns

Index(['index_left', 'inspection_date', 'inspector', 'number_of_violations',
       'violations', 'zip_code', 'house_numbers', 'categories', 'distance',
       'review_count', 'longitude', 'latitude', 'name', 'sector', 'district',
       'total_score', 'rating', 'health_scores', 'sector_encode',
       'index_right', 'sector_0', 'sector_1', 'sector_2', 'sector_3',
       'sector_4', 'sector_5', 'sector_6', 'sector_7', 'sector_8', 'sector_9',
       'sector_10', 'sector_11', 'sector_12', 'sector_13', 'sector_14',
       'sector_15', 'sector_16', 'sector_17', 'sector_18', 'sector_19',
       'sector_20', 'sector_21', 'sector_22', 'sector_23', 'sector_24',
       'sector_25', 'sector_26', 'sector_27', 'sector_28', 'sector_29',
       'sector_30', 'sector_31', 'sector_32', 'sector_33', '1', '2', '3', '4',
       'All', 'less_92', '92_to_95', '96_to_99', '100'],
      dtype='object')

In [29]:
df = df[['name','review_count','rating','less_92','92_to_95','96_to_99','100', 'health_scores','sector_0', 'sector_1', 'sector_2', 'sector_3',
       'sector_4', 'sector_5', 'sector_6', 'sector_7', 'sector_8', 'sector_9',
       'sector_10', 'sector_11', 'sector_12', 'sector_13', 'sector_14',
       'sector_15', 'sector_16', 'sector_17', 'sector_18', 'sector_19',
       'sector_20', 'sector_21', 'sector_22', 'sector_23', 'sector_24',
       'sector_25', 'sector_26', 'sector_27', 'sector_28', 'sector_29',
       'sector_30', 'sector_31', 'sector_32', 'sector_33']]
df.head()

,name,review_count,rating,less_92,92_to_95,96_to_99,100,health_scores,sector_0,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,sector_10,sector_11,sector_12,sector_13,sector_14,sector_15,sector_16,sector_17,sector_18,sector_19,sector_20,sector_21,sector_22,sector_23,sector_24,sector_25,sector_26,sector_27,sector_28,sector_29,sector_30,sector_31,sector_32,sector_33
0,Capparelli's On Main,99.0,3.5,0.3,0.32,0.23,0.15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Fina's Kitchen,42.0,4.0,0.3,0.32,0.23,0.15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Rolando's Super Tacos No 1,145.0,3.5,0.3,0.32,0.23,0.15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Rolando's Super Tacos No 1,145.0,3.5,0.3,0.32,0.23,0.15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Fina's Kitchen,42.0,4.0,0.3,0.32,0.23,0.15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Splitting the Data

In [30]:
#training / test split
X= df.drop(columns=['name','health_scores'])
y= df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=df.health_scores)
X_train.head()

,review_count,rating,less_92,92_to_95,96_to_99,100,sector_0,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,sector_10,sector_11,sector_12,sector_13,sector_14,sector_15,sector_16,sector_17,sector_18,sector_19,sector_20,sector_21,sector_22,sector_23,sector_24,sector_25,sector_26,sector_27,sector_28,sector_29,sector_30,sector_31,sector_32,sector_33
2146,46.0,3.0,0.56,0.18,0.26,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540,3.0,2.5,0.31,0.22,0.27,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1064,315.0,4.0,0.28,0.26,0.29,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1172,304.0,4.0,0.33,0.35,0.29,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1087,16.0,3.5,0.28,0.26,0.29,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
#Train Model

#Create the Decision Tree Object

clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state= 42, min_samples_split= 2)

In [32]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [33]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

array(['4', '2', '3', ..., '4', '4', '4'], dtype=object)

In [34]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.07117438, 0.20640569, 0.29893238, 0.42348754],
       [0.1959799 , 0.3718593 , 0.17085427, 0.26130653],
       [0.1372549 , 0.24836601, 0.31372549, 0.30065359],
       ...,
       [0.07117438, 0.20640569, 0.29893238, 0.42348754],
       [0.07117438, 0.20640569, 0.29893238, 0.42348754],
       [0.09302326, 0.23255814, 0.30232558, 0.37209302]])

In [35]:
confusion_matrix(y_train, y_pred)

array([[140,  87,  43,  30],
       [ 71, 237,  73,  80],
       [ 31, 121, 114, 143],
       [ 25,  95,  74, 250]])

In [36]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.46


In [37]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.41


In [38]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.52      0.47      0.49       300
           2       0.44      0.51      0.47       461
           3       0.38      0.28      0.32       409
           4       0.50      0.56      0.53       444

    accuracy                           0.46      1614
   macro avg       0.46      0.46      0.45      1614
weighted avg       0.45      0.46      0.45      1614



### Creating a Random Forrest Model and setting the estimator to one million, because why not

In [ ]:
#Train Model

#Create the Random Forest Object
#
rf = RandomForestClassifier(max_depth=4, criterion='entropy', n_estimators=100000, random_state=42)
#Fit the model to the training data
rf.fit(X_train, y_train)

In [ ]:
#Print the Feature importance
print(rf.feature_importances_)

In [ ]:
#Estimate the probability of the health group, using the training data
rf.score(X_train, y_train)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
#Compute the accuracy of the model when run on the test data

print('Accuracy of random forest classifier on test set: {:.2f}'
     .format(rf.score(X_test, y_test)))

# Creating a Confusion Matrix To Better Understand the Decision Tree Model

In [40]:
y_pred = clf.predict(X_test)
y_pred

array(['2', '2', '3', '2', '4', '3', '3', '1', '4', '3', '2', '3', '1',
       '1', '2', '4', '4', '1', '2', '2', '1', '3', '1', '4', '2', '2',
       '2', '4', '4', '2', '2', '3', '2', '4', '4', '2', '2', '2', '2',
       '4', '3', '2', '2', '4', '4', '3', '4', '1', '4', '1', '4', '4',
       '1', '2', '3', '2', '3', '1', '3', '2', '1', '3', '3', '2', '1',
       '2', '3', '4', '3', '2', '4', '4', '2', '1', '2', '2', '2', '3',
       '3', '2', '4', '2', '2', '3', '3', '2', '2', '2', '4', '2', '2',
       '2', '4', '4', '2', '4', '2', '3', '1', '4', '1', '4', '2', '1',
       '2', '2', '2', '3', '2', '2', '2', '2', '4', '4', '1', '4', '3',
       '2', '4', '1', '1', '4', '3', '3', '2', '4', '1', '1', '3', '2',
       '3', '4', '3', '4', '2', '3', '1', '3', '4', '4', '4', '2', '3',
       '3', '3', '3', '4', '4', '4', '3', '4', '4', '3', '3', '2', '2',
       '4', '3', '2', '3', '3', '4', '2', '2', '4', '2', '4', '4', '4',
       '2', '1', '4', '2', '1', '3', '4', '4', '2', '2', '3', '3

In [41]:
confusion_matrix(y_test,y_pred)

array([[54, 35, 26, 13],
       [30, 95, 29, 44],
       [12, 69, 40, 55],
       [13, 49, 31, 98]])

In [43]:
#Making the confusion matrix "pretty"
labels=(" 100 ", " 96-99 ", " 92-95 ", " <92 ")
pretty_cm=pd.DataFrame(confusion_matrix(y_test,y_pred),index=labels,columns=labels)
pretty_cm.T

,100,96-99,92-95,<92
100,54,30,12,13
96-99,35,95,69,49
92-95,26,29,40,31
<92,13,44,55,98


## Takeaway: There is correlation between inspectors and the scores they give.
### Further exploration is warranted. The distribution of missed predictions highlights one of the drawbacks to the model, which was the inconsistency of restaurant categorization of each individual restaurant in Yelp Fusion. Having the categories encoded and used as features would produce an improved model. 
